# Ayudantía virtual: Decoradores

Puedes correr el código de esta ayudantía usando Jupyter Notebook. Si tienes alguna duda respecto a ésta, pregúntala en las [issues](https://github.com/IIC2233/Syllabus/issues).

En esta ayudantía, resolveremos paso a paso la Actividad 6 del primer semestre del año 2016. Para esto, asegúrate primero de haber leído el [material de esta semana](https://github.com/IIC2233/contenidos/tree/master/semana-7).

Por favor, lee el enunciado entero de la actividad [aquí](https://github.com/IIC2233-2016-1/syllabus/blob/master/Actividades/AC06/main.pdf) antes de continuar.

> En adelante, cuando nos refiramos a una parte del enunciado, lo mostraremos en un bloque de texto como este.

## Solución AC06 2016-1: Decoradores

> ### Instrucciones

>A pesar de su experiencia trabajando en MercadoPreso para Bucchi Incorporated, usted sigue como
empleado de dicha empresa. Como habrá notado, el ambiente es medio rígido. En esta ocasión, debe coordinar
el trabajo de dos colegas. El primero le entregó las clases y funciones que van a usar en el proyecto. El segundo
se encargó de hacer la ejecución. Como usted se maneja a la perfección con las buenas prácticas de PEP8,
nota que el código que le pasaron está bien, pero no es perfecto. Como los programadores que lo hicieron
son mañosos, no toleran **ningún cambio a alguna línea que hayan escrito**, por lo que usted deberá **corregir el
programa sin modificar las clases o las funciones**.

> ### Requerimientos

>En el enunciado se nos piden programar y aplicar decoradores que: 

> > 1) Verifique si los métodos dados, de la clase Producto, existen y, en caso de que estén en CamelCase transformelos a snake_case de forma que cumplan con PEP8.

> > 2) Convierta métodos específicos de la clase Producto en métodos privados.
    
> > 3) Corrija el IVA en el precio final del producto, para que se aplique un 19 %, en vez del que especificó el otro programador.

> > 4) Evite que al incrementar el precio del producto se entregue un valor que no sea del tipo int.
    

> ### To do

>Los requisitos específicos se listan a continuación: 

> > * R1: Crear el decorador `snake_case` que reciba una cantidad no determinada de métodos, verifique si estos existen y, si estos existen, detecte si están en CamelCase y transfórmelos a snake case.
        
> > * R2: Crear el decorador `protect_methods` que reciba una cantidad no determinada de métodos y que haga privados los métodos de la clase que debiesen serlo.

> > * R3: Crear el decorador `correct_price` que reciba el porcentaje de IVA dado en el programa y el porcentaje correcto a utilizar y que corrija el cálculo del precio de un producto.

> > * R4: Crear el decorador `check_input_is` que reciba una cantidad no determinada de tipos y que verifique que los tipos de los argumentos que recibe un método sean del tipo esperado.

> > * R5: Decorar la clase y los métodos con los decoradores y argumentos que correspondan.
    

> ### Tips

> > Para extraer, borrar o escribir algún método de una clase puedes usar las siguientes funciones

> > * `getattr(cls, "nombre metodo")`

> > * `delattr(cls, "nombre metodo")`

> > * `setattr(cls, "nombre metodo", metodo)`

En primer lugar, debido a que se nos entrega un código que no podemos modificar, usaremos decoradores para alterar el comportamiento de métodos o clases específicos. El primer objetivo será familiarizarnos con el código original que se nos entregó para poder identificar dos aspectos claves:

* Los métodos o clases que se deben decorar para modificar su comportamiento.
* El tipo de decorador que debemos crear para poder cumplir los requerimientos. Con respecto a sus argumentos, el decorador puede recibir o no recibir parámetros y con respecto a quién decora, puede decorar una clase o un método.

A continuación se presenta el código base entregado.

In [1]:
class Product:

    def __init__(self, name, price, stock):
        self.name = name
        self._price = price
        self._stock = stock

    def IncreasePrice(self, amount):
        self._price += amount

        
    def sell(self, amount=1):
        self._stock -= amount
        print("Se vendieron {0} productos en ${1}"
              .format(amount, amount * self.final_price))
    
    def ChangePrice(self, new_price):
        print("Has accedido a un metodo que debiese ser privado")
        self._price = new_price

    @property
    def FinalPrice(self):
        return self._price * 1.08


if __name__ == "__main__":
    ej = Product("Auto", 1000000, 100)
    print("IncreasePrice: 'hola'(str)")
    ej.increase_price('hola')
    print("Precio: {}".format(ej.final_price))
    print()

    print("IncreasePrice: 100(int)")
    ej.increase_price(100)
    print("Precio: {}".format(ej.final_price))

IncreasePrice: 'hola'(str)


AttributeError: 'Product' object has no attribute 'increase_price'

### Requisito 1: verificación de PEP8

El decorador debe recibir un número _indeterminado_ de métodos de la clase `Producto`. Esto nos advierte que no podemos asumir el número de métodos a verificar y por lo tanto tendremos que recurrir al empleo de argumentos posicionales `*args`. Debido a que el decorador requiere acceso a todos los métodos de la clase para poder verificar si sus argumentos son métodos que deben modificarse, entonces este `snake_case` debe decorar la clase `Producto`. Con esto, la estructura básica del decorador `snake_case` será:

In [3]:
def snake_case(*args): # este es el constructor del decorador, que recibe los nombres de los métodos a verificar
    def decorador(cls): # este es el decorador propiamente tal, que recibe la clase a modificar
        # aquí se modifica la clase
        pass
        return cls
    return decorador

Antes de construir el decorador, cabe notar que el nombre `args` no es fijo y en el contexto de argumentos posicionales se puede usar cualquier otro nombre con un arterisco antepuesto. Para enfatizar esto, en este decorador usaremos `*metodos` para denotar los n argumentos que se le entregarán como argumento.

Ahora bien, para conseguir el objetivo del decorador `snake_case`, primero debemos verificar si los argumentos entregados son métodos de la clase. Para esto se utiliza la función `hasattr(cls, method)` que retorna `True` si la clase `cls` tiene el método de nombre `method` y `False` en caso contrario. Luego, si el método pertenece a la clase y además contiene mayúsculas, entonces se debe llevar a snake case, quitando las mayúsculas y agregando un _underscore_ delante de las mayúsculas no iniciales. Con lo anterior, el decorador resultante es:

In [4]:
def snake_case(*metodos):
    def decorador(cls):
        for metodo in metodos:
            if hasattr(cls, metodo) and metodo.lower() != metodo:
                funcionalidad = getattr(cls, metodo) # rescatamos el método (funcionalidad) asociado al nombre entregado
                nuevo_nombre = metodo[0].lower()
                for letra in metodo[1:]:
                    if letra.isupper():
                        nuevo_nombre += "_"+letra.lower()
                    else:
                        nuevo_nombre += letra
                delattr(cls, metodo) # borramos el método con el nombre original
                setattr(cls, nuevo_nombre, funcionalidad) # asociamos la funcionalidad original al nuevo nombre
        return cls
    return decorador

Es importante notar que cualquier **constructor de decoradores** debe retornar el decorador, mientras que cualquier **decorador** debe retornar el objeto modificado. De lo contrario no se estarán aplicando las modificaciones pertinentes.

### Requisito 2: métodos internos

El decorador debe recibir un número arbitrario de nombres de métodos y debe anteponer un doble _underscore_ a cada uno. A diferencia del requisito anterior, aquí no se exige verificar que el método pertenezca a la clase (se asume que no se entregarán métodos inexistentes). Debido a que también necesita acceso a la clase para poder asociar nuevos nombres con funcionalidades preexistentes, es necesario que este decorador decore la clase. Con todo esto, una posible implementación es:

In [5]:
def protect_methods(*metodos): # constructor que recibe nombres de métodos
    def decorador(cls): # decorador de una clase
        for metodo in metodos:
            funcionalidad = getattr(cls, metodo)
            nuevo_nombre = "__{}".format(metodo)
            setattr(cls, nuevo_nombre, funcionalidad) # asignamos nombre
            delattr(cls, metodo) # borramos el anterior
        return cls
    return decorador

### Requisito 3: corrección de precios

El decorador debe recibir dos números que indican el porcentaje de IVA antes y después de la modificación. Se requiere que este cambio se efectúa a nivel del cálculo del precio de un producto, de forma que se debe decorar la _property_ `FinalPrice` (o `final_price` luego de llevarla a snake case con el primer decorador). Como debemos recibir parámetros y se decora un método, entonces la estructura general de `correct_price` es:

In [6]:
def correct_price(iva_actual, iva_nuevo): # constructor del decorador, recibe los IVA
    def decorador(metodo): # decorador que recibe el método a modificar
        def nuevo_metodo(*args, **kwargs): # nuevo método que reemplazará al antiguo
            # aquí se modifica el comportamiento
            pass
        return metodo
    return decorador

¿Cómo entendemos los argumentos de la función `nuevo_método`? Tanto `*args` como `**kwargs` denotan los argumentos necesarios para llamar a `metodo`. Es importante notar que `nuevo_metodo` debe tener **como mínimo** estos argumentos.

Para recalcular el precio, notemos que el precio original (sin IVA) es el retorno de `metodo` dividido por el IVA antiguo. Luego, el precio modificado será el precio original calculado multiplicado por el IVA nuevo. Con esto, el decorador nos queda según:

In [7]:
def correct_price(iva_actual, iva_nuevo): 
    def decorador(metodo): 
        def nuevo_metodo(*args, **kwargs): 
            precio_original = metodo(*args, **kwargs) / iva_actual
            precio_nuevo = precio_original * iva_nuevo
            return precio_nuevo
        return nuevo_metodo
    return decorador

### Requisito 4: tipos de argumentos

El decorador debe recibir un número indeterminado de tipos de dato que deberán ser contrastados con los argumentos de un método. Como esta comparación depende del método en específico, se realizará con un decorador de métodos y no de clases. Con esto, la estructura será:

In [8]:
def check_input_is(*tipos): # constructor con tipos de datos
    def decorador(metodo): # decorador que recibe el método
        def interno(*args): # manejo de la verificación
            # modificacion 
            pass
        return interno
    return decorador

Para realizar la verificación, se deben tomar los argumentos de `metodo`, omitir el primero de ellos (pues corresponde a una referencia al objeto mismo o `self`) y comparar uno a uno con el tipo de dato entregado como argumento al constructor. Cabe notar que en este caso la función `interno` solo recibe argumentos posicionales porque se realizará esta comparación de tipos uno a uno con los tipos entregados. De esta forma, el resultado es: 

In [9]:
def check_input_is(*tipos): 
    def decorador(metodo):
        def interno(*args):
            tipos_recibidos = tuple(type(a) for a in args)[1:] # omitimos el primero
            if tipos_recibidos == tipos:
                return metodo(*args)
            print("Argumentos no son del tipo esperado!!!")
        return interno
    return decorador

Antes de concluir, notemos como los dos últimos decoradores comparten que ambos decoran métodos, pero difieren levemente en la modificación que realizan. En `correct_price` se retorna siempre un número que reemplazará al retorno original del método decorado, mientras que en `check_input_is` solo se retorna cuando se cumple el criterio de tipos, en cuyo caso se retorna el valor original del método decorado.

### Requisito 5: aplicando los decoradores

Para finalizar, solo basta incorporar los decoradores de forma adecuada al código entregado. Usando el azúcar sintáctico del material, el decorador se coloca justo arriba de la clase o método a decorar, incluyendo los argumentos necesarios. En caso que haya más de un decorador en un mismo objeto, se aplican partiendo desde el más cercano al mismo; así, el que queda primero en términos de líneas de código se aplica sobre el resultado de aplicar los demás decoradores. El código final, con la declaración de los decoradores y el código base modificado se muestra a continuación.

In [2]:
# -------------------------------------------------------------------------------------------------------------------- #

#                                                   DECORADORES                                                        #

# -------------------------------------------------------------------------------------------------------------------- #

def snake_case(*metodos):
    def decorador(cls):
        for metodo in metodos:
            if hasattr(cls, metodo) and metodo.lower() != metodo:
                funcionalidad = getattr(cls, metodo) # rescatamos el método (funcionalidad) asociado al nombre entregado
                nuevo_nombre = metodo[0].lower()
                for letra in metodo[1:]:
                    if letra.isupper():
                        nuevo_nombre += "_"+letra.lower()
                    else:
                        nuevo_nombre += letra
                delattr(cls, metodo) # borramos el método con el nombre original
                setattr(cls, nuevo_nombre, funcionalidad) # asociamos la funcionalidad original al nuevo nombre
        return cls
    return decorador


def protect_methods(*metodos): # constructor que recibe nombres de métodos
    def decorador(cls): # decorador de una clase
        for metodo in metodos:
            funcionalidad = getattr(cls, metodo)
            nuevo_nombre = "__{}".format(metodo)
            setattr(cls, nuevo_nombre, funcionalidad) # asignamos nombre
            delattr(cls, metodo) # borramos el anterior
        return cls
    return decorador


def correct_price(iva_actual, iva_nuevo): 
    def decorador(metodo): 
        def nuevo_metodo(*args, **kwargs): 
            precio_original = metodo(*args, **kwargs) / iva_actual
            precio_nuevo = precio_original * iva_nuevo
            return precio_nuevo
        return nuevo_metodo
    return decorador


def check_input_is(*tipos): 
    def decorador(metodo):
        def interno(*args):
            tipos_recibidos = tuple(type(a) for a in args)[1:] # omitimos el primero
            if tipos_recibidos == tipos:
                return metodo(*args)
            print("Argumentos no son del tipo esperado!!!")
        return interno
    return decorador




# -------------------------------------------------------------------------------------------------------------------- #

#                                                   CODIGO ORIGINAL                                                    #

# -------------------------------------------------------------------------------------------------------------------- #


@protect_methods("change_price")
@snake_case("IncreasePrice", "FinalPrice", "ChangePrice")
class Product:

    def __init__(self, name, price, stock):
        self.name = name
        self._price = price
        self._stock = stock

    @check_input_is(int)
    def IncreasePrice(self, amount):
        self._price += amount

    def sell(self, amount=1):
        self._stock -= amount
        print("Se vendieron {0} productos en ${1}"
              .format(amount, amount * self.final_price))

    def ChangePrice(self, new_price):
        print("Has accedido a un metodo que debiese ser privado")
        self._price = new_price


    @property
    @correct_price(1.08, 1.19)
    def FinalPrice(self):
        return self._price * 1.08


if __name__ == "__main__":
    ej = Product("Auto", 1000000, 100)
    print("IncreasePrice: 'hola'(str)")
    ej.increase_price('hola')
    print("Precio: {}".format(ej.final_price))
    print()

    print("IncreasePrice: 100(int)")
    ej.increase_price(100)
    print("Precio: {}".format(ej.final_price))


IncreasePrice: 'hola'(str)
Argumentos no son del tipo esperado!!!
Precio: 1189999.9999999998

IncreasePrice: 100(int)
Precio: 1190118.9999999998
